In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

prefs = {
    "download.default_directory": r"/content/drive/MyDrive/Stage_Med_Reg(Badji)/protocol",
    "download.prompt_for_download": False,  # Désactive l'invite de téléchargement
    "directory_upgrade": True               # S'assure que le répertoire est créé ou mis à jour
}

# Ajout des préférences expérimentales à chrome_options
chrome_options.add_experimental_option('prefs', prefs)

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
url = "put-url-here-to-scrape"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [7]:
driver.get('https://www.clinicaltrialsregister.eu/ctr-search/search')

In [8]:
import pandas as pd

medecines = pd.read_csv('/content/medecine.csv')
medecines

,Unnamed: 0,ProductName
0,0,Leflunomide medac
1,1,Filgrastim Hexal
2,2,Qutenza
3,3,Levitra
4,4,Rilutek
...,...,...
885,885,Prandin
886,886,Litak
887,887,NovoNorm
888,888,Foscan


In [ ]:
protocols = []
for name in medecines['ProductName'][252:]:
  driver.find_element("id", "query").clear()
  inputElement = driver.find_element("id", "query")
  inputElement.send_keys(name)
  time.sleep(1)
  inputElement.submit()
  time.sleep(2)
  usa = driver.find_element('xpath', '//*[@id="tabs-1"]')
  driver.execute_script('arguments[0].scrollIntoView(true)', usa)

  liste = driver.find_elements("css selector", "#tabs-1 > div.results.grid_8plus")
  for e in liste:
      trials = e.find_elements("css selector", '#tabs-1 > div.results.grid_8plus > table:nth-child(1) > tbody > tr:nth-child(7) > td > a')
      for trial in trials:
         protocols.append(trial.get_attribute('href'))
    #next = driver.find_element('xpath', '//*[@id="tabs-1"]/div[2]/a[9]')
    #next.click()
  driver.execute_script("window.scrollBy(0,0)","")

In [10]:
protocols

['https://www.clinicaltrialsregister.eu/ctr-search/trial/2006-004455-37/DE',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/SE',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/ES',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/PT',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/FR',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/IT',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/GR',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/NL',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/DE',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/DK',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/BG',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2020-005269-15/PL',
 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2019-000199-41/ES']

In [ ]:
len(protocols)

1081

In [ ]:
from fpdf import FPDF
from selenium.webdriver.common.by import By
from PyPDF2 import PdfReader
import re
import os

download_dir = '/content/drive/MyDrive/Stage_Med_Reg(Badji)/protocol'
for protocol in protocols:
    driver.get(protocol)
    download = driver.find_element(By.XPATH, '//*[@id="downloadCTA"]')
    download.click()
    time.sleep(5)

    # Renommer le fichier téléchargé
    original_filename = "trial.txt"  # Remplacez par le nom exact du fichier
    original_filepath = os.path.join(download_dir, original_filename)
    match = re.search(r'trial/(\d{4}-\d{6}-\d{2})/(\w{2})', protocol)
    new_filename = os.path.join(download_dir, f"{match.group(1)}_{match.group(2)}.txt")
    os.rename(original_filepath, new_filename)

    time.sleep(5)

    print(f"File saved as: {new_filename}")

driver.quit()


In [ ]:
def convert_txt_to_pdf(txt_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(txt_dir):
        if filename.endswith(".txt"):
            txt_file_path = os.path.join(txt_dir, filename)
            output_pdf_path = os.path.join(output_dir, f"{filename[:-4]}.pdf")

            # Read the text file content
            with open(txt_file_path, "r", encoding="utf-8") as f:
                text = f.read()

            # Create a new PDF instance
            pdf = FPDF()

            # Add a new page
            pdf.add_page()

            # Set the font
            pdf.set_font("Arial", size=12)

            # Add the text content to the PDF
            for line in text.splitlines():
                sanitized_text = sanitize_text(line.strip())  # Apply any necessary text sanitization
                pdf.cell(200, 10, txt=sanitized_text, ln=1, align="L")

            # Save the PDF file
            pdf.output(output_pdf_path)

            print(f"Converted: {filename} to {output_pdf_path}")

In [ ]:
txt_dir = "C:/Users/dell/Desktop/trialProtocols/"  # Replace with your actual directory
output_dir = "C:/Users/dell/Desktop/trialProtocols/trialspdf"  # Replace with your desired output directory

convert_txt_to_pdf(txt_dir, output_dir)

Converted: 2004-000020-32_BE.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000020-32_BE.pdf
Converted: 2004-000088-92_ES.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000088-92_ES.pdf
Converted: 2004-000088-92_FI.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000088-92_FI.pdf
Converted: 2004-000088-92_IT.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000088-92_IT.pdf
Converted: 2004-000356-17_AT.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000356-17_AT.pdf
Converted: 2004-000356-17_DK.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000356-17_DK.pdf
Converted: 2004-000356-17_ES.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000356-17_ES.pdf
Converted: 2004-000356-17_FI.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000356-17_FI.pdf
Converted: 2004-000356-17_IT.txt to C:/Users/dell/Desktop/trialProtocols/trialspdf\2004-000356-17_IT.pdf
Converted: 2004-000356-17_SE.txt to C:/Users/dell/Deskt